In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
from tcn import TCN, tcn_full_summary
from tcn import compiled_tcn
from mango.tuner import Tuner
from keras_flops import get_flops
from hardware_utils import *
import pickle
import csv
import time
import itertools
import random
import get_dataset as kws_data
import kws_util
import argparse

In [ ]:
EPOCHS = 60 #model epochs
NAS_EPOCHS = 50 #NAS epochs
kws_data_dir = 'kws_data/' #cifar-10-dataset-directory
device = "NUCLEO_L4R5ZI_P" #hardware name
dirpath='KWS_Mbed_Prog/' #mbed program directory
model_name = "kwsmodel_"+device+"_"+".h5"
platform_connected = True #HIL or proxy
quantization = True #use quantization
os.system("mkdir -p kws_data/")
os.system("mkdir -p trained_models/")

log_file_name = 'log_NAS_kws'+str(platform_connected)+"_"+device+'.csv' #log file for NAS
if os.path.exists(log_file_name):
    os.remove(log_file_name)
row_write = ['score', 'accuracy','SRAM','Flash','Latency',
             'nb_filters','kernel_size','dilations','nb_stacks','use_skip_connections']
with open(log_file_name, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_write)
if os.path.exists(log_file_name[0:-4]+'.p'):
    os.remove(log_file_name[0:-4]+'.p')

## Import Dataset

In [ ]:
Flags, unparsed = kws_util.parse_command()
Flags.data_dir = kws_data_dir
print('We will download data to {:}'.format(Flags.data_dir))
ds_train, ds_test, ds_val = kws_data.get_training_data(Flags)
print("Done getting data")
train_shuffle_buffer_size = 85511
val_shuffle_buffer_size = 10102
test_shuffle_buffer_size = 4890

ds_train = ds_train.shuffle(train_shuffle_buffer_size)
ds_val = ds_val.shuffle(val_shuffle_buffer_size)
ds_test = ds_test.shuffle(test_shuffle_buffer_size)

## Training and NAS

In [ ]:
window_size_samples = int(Flags.sample_rate* Flags.window_size_ms / 1000)
window_stride_samples = int(Flags.sample_rate * Flags.window_stride_ms / 1000)
desired_samples = int(Flags.sample_rate * Flags.clip_duration_ms / 1000)
length_minus_window = (desired_samples - window_size_samples)
spectrogram_length = 1 + int(length_minus_window / window_stride_samples)

In [ ]:
def objective_NN(nb_filters=6,kernel_size=32,dilations=[1,2,4,8,16,32,64,128],
                 nb_stacks=1, use_skip_connections = True, platform_connected = False, quantization=True):
    print(nb_filters,kernel_size,dilations,nb_stacks, use_skip_connections)
    training_flag = 0
    score = -5.0
    accuracy = -1.0
    ################################################################################
    inputs = layers.Input((spectrogram_length,Flags.dct_coefficient_count,1))
    x = inputs
    x = layers.Reshape((spectrogram_length,Flags.dct_coefficient_count))(x)
    x = TCN(return_sequences=False,
                     nb_filters=nb_filters,
                     kernel_size=kernel_size,
                     dilations=dilations,
                     nb_stacks=nb_stacks,
                     use_weight_norm=False,
                     use_skip_connections=use_skip_connections)(x)
    outputs = layers.Dense(12, activation="softmax", name="pred")(x)
    model = Model(inputs = inputs, outputs = outputs)
    ################################################################################
    callbacks = kws_util.get_callbacks(args=Flags)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
    losses = tf.keras.losses.SparseCategoricalCrossentropy()
    metrics = tf.keras.metrics.SparseCategoricalAccuracy()
    model.compile(optimizer=optimizer, loss=losses, metrics=metrics)
    maxRAM, maxFlash = return_hardware_specs(device)
    if(platform_connected == False):
        Latency = get_flops(model, batch_size=1) #latency proxy
        RAM = get_model_memory_usage(batch_size=1, model=model) #SRAM proxy
        if(quantization==True):
            RAM = RAM/8.0
        Flash =  get_model_flash_usage(model,'g_kws_model_data',quantization=quantization) #flash proxy
        if(RAM < maxRAM and Flash < maxFlash):
            training_flag = 1
        else:
            training_flag = 0
            score = -5.0
            accuracy = -1.0
            
    if(training_flag == 1):
        history = model.fit(ds_train, validation_data=ds_val, epochs=EPOCHS, callbacks=callbacks)
        accuracy = history.history['val_sparse_categorical_accuracy'][-1]
        if(platform_connected == False):
            score = accuracy + 0.01*((RAM/maxRAM) + (Flash/maxFlash)) +  0.01*(Latency/1e6)
        else:
            score = accuracy + 0.01*((RAM/maxRAM) + (Flash/maxFlash)) +  0.01*(Latency/0.8)
    
    row_write = [score, accuracy,RAM,Flash,Latency,
             nb_filters,kernel_size,dilations,nb_stacks,use_skip_connections]
    print('Design choice:',row_write)
    with open(log_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_write)     
    
    return score

In [ ]:
def save_res(data, file_name):
    pickle.dump( data, open( file_name, "wb" ) )
    
min_layer = 3
max_layer = 8
a_list = [1,2,4,8,16,32,64,128,256]
all_combinations = []
dil_list = []
for r in range(len(a_list) + 1):
    combinations_object = itertools.combinations(a_list, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list
all_combinations = all_combinations[1:]
for item in all_combinations:
    if(len(item) >= min_layer and len(item) <= max_layer):
        dil_list.append(list(item))

param_dict = {
    'nb_filters': range(2,64),
    'nb_stacks': [1,2,3],
    'kernel_size': range(2,16),
    'use_skip_connections': [True, False],
    'dilations': dil_list
}

def objfunc(args_list):

    objective_evaluated = []
    
    start_time = time.time()
    
    for hyper_par in args_list:
        nb_filters = hyper_par['nb_filters']
        nb_stacks = hyper_par['nb_stacks']
        kernel_size = hyper_par['kernel_size']
        use_skip_connections = hyper_par['use_skip_connections']
        dilations = hyper_par['dilations']
            
        objective = objective_NN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,
                 nb_stacks=nb_stacks, use_skip_connections = True, 
                                 platform_connected = platform_connected, quantization=quantization)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        
    return objective_evaluated

In [ ]:
conf_Dict = dict()
conf_Dict['batch_size'] = 1 
conf_Dict['num_iteration'] = NAS_EPOCHS
conf_Dict['initial_random']= 5
tuner = Tuner(param_dict, objfunc,conf_Dict)
all_runs = []
results = tuner.maximize()
all_runs.append(results)
save_res(all_runs,log_file_name[0:-4]+'.p')

## Train and evaluate the best model

In [ ]:
nb_filters = results['best_params']['nb_filters']
nb_stacks = results['best_params']['nb_stacks']
kernel_size = results['best_params']['kernel_size']
use_skip_connections = results['best_params']['use_skip_connections']
dilations = results['best_params']['dilations']
################################################################################
inputs = layers.Input((spectrogram_length,Flags.dct_coefficient_count,1))
x = inputs
x = layers.Reshape((spectrogram_length,Flags.dct_coefficient_count))(x)
x = TCN(return_sequences=False,
                 nb_filters=nb_filters,
                 kernel_size=kernel_size,
                 dilations=dilations,
                 nb_stacks=nb_stacks,
                 use_weight_norm=False,
                 use_skip_connections=use_skip_connections)(x)
outputs = layers.Dense(12, activation="softmax", name="pred")(x)
model = Model(inputs = inputs, outputs = outputs)
################################################################################
callbacks = kws_util.get_callbacks(args=Flags)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
losses = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.keras.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)
history = model.fit(ds_train, validation_data=ds_val, epochs=EPOCHS, callbacks=callbacks)

In [ ]:
model.save("trained_models/" + model_name)

In [ ]:
test_scores = model.evaluate(ds_test)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])